In [69]:
from stock_env.envs.single_stock_env import SingleStockEnv
import numpy as np
import pandas as pd
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.dqn import DQN
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
price = np.load('price.pkl.npy')
df = pd.DataFrame()
df['A'] = price
env = SingleStockEnv(
    df=df,
    init_cash=5e4,
)
check_env(env)

model = DQN.load("log/dqn_single_stock", env=env)

In [ ]:
done = False
obs = env.reset()
while not done:
    action, _ = model.predict(obs, deterministic=True)
    print(action)
    obs, _, done, _ = env.step(action)

data = pd.concat([df, pd.DataFrame(env.history)], axis=1, join='inner')

In [29]:
data = pd.read_csv('history.csv', index_col=0)
data.head()

,A,actions,delta_shares,quantity,delta_vt,total_reward,total_profit,portfolio_value,nav,cash
0,50.000000,3,-200,-200,-84.978266,-85.339331,-84.978266,49915.021734,-10024.978266,59940.000000
1,50.124891,3,-200,-400,-71.348092,-156.941950,-156.326357,49843.673643,-20061.304623,69904.978266
2,50.153262,3,-200,-600,-101.677322,-259.136186,-258.003679,49741.996321,-30133.634256,79875.630577
3,50.222724,3,-200,-800,-187.523964,-448.418412,-445.527643,49554.472357,-40305.702972,89860.175329
4,50.382129,3,-200,-1000,-193.228905,-643.514188,-638.756549,49361.243451,-50515.357621,99876.601072


In [40]:
returns = data['portfolio_value'].pct_change()
returns.index = pd.date_range("2018-01-01", periods=4999, freq="D").to_pydatetime()
returns = returns.dropna()